In [22]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from functionsMaths import calculateDelta, get_mse,gradiente_descendente, feedforward, div_tuplas
import os
import math
from statistics import mean
from Distorsionador import Distorsionador

In [5]:
#Inicializamos los pesos de la Capa 1
w = np.random.random(500)
w11 = w[:100]
w12 = w[100:200]
w13 = w[200:300]
w14 = w[300:400]
w15 = w[400:500]
W1=[w11,w12,w13,w14,w15]


#Inicializamos los pesos de la Capa 2
w = np.random.random(25)
w21 = w[:5]
w22 = w[5:10]
w23 = w[10:15]
w24 = w[15:20]
w25 = w[20:25]
W2 = [w21,w22,w23,w24,w25] 


#Incializamos los pesos de la Capa 3
w = np.random.random(15)
w31 = w[:5]
w32 = w[5:10]
w33 = w[10:15]
W3 = [w31,w32,w33]

#Conversiones
W1 = np.array(W1)
W2 = np.array(W2)
W3 = np.array(W3)
W = [W1,W2,W3]


#Bias
b = [random.random() for i in range(13)]
b1= b[:5]
b2 = b[5:10]
b3 = b [10:]
B = [b1,b2,b3]

#Factor de aprendizaje
alfa = np.array(random.random())
beta = np.array(random.random())

#Codificacion de clases
#b = 100
#d = 010
#f = 001

In [45]:
cantidad =['100','500','1000']
set_validation = [0.1,0.2,0.3]

In [46]:
for set in range (len(cantidad)):
    for percentage in range(len(set_validation)):
        
     data = (pd.read_csv(os.path.join(os.path.abspath(''),"data","distorsionadas",cantidad[set],'letras.csv'),sep=';',header=None)).to_numpy()
     #Convertimos el dataframe en array numpy
     data_train = data[:int(len(data)-int(len(data)*set_validation[percentage])*2)]
     data_validation = data[int(len(data)-int(len(data)*set_validation[percentage])*2):int(len(data)-int(len(data)*set_validation[percentage])*2)+int(len(data)*set_validation[percentage])]
     data_test = data[int(len(data)-int(len(data)*set_validation[percentage])*2)+int(len(data)*set_validation[percentage]):]

     #Division de datos en tuplas(datos,clase)
     letras_train = div_tuplas(data_train)
     letras_test = div_tuplas(data_train)
     letras_validation = div_tuplas(data_validation)

     tabla = pd.DataFrame(epocas)

     #Comienzo de Perceptron
     epocas =1
     err_train_epoc = []
     err_valid_epoc = []

     for epoc in range(epocas):
        err_training = np.array([0,0,0])
        err_validation = np.array([0,0,0])
    
        for i in range(len(letras_train)):
        
         #TRAINING
         Y = feedforward(letras_train[i][0],W,B)
         Ye = np.array(letras_train[i][1])
         err_training = err_training + ((Ye-Y[-1])**2)

         #Deltas
         deltas = calculateDelta(Y[-1],Ye,W[1:])

         W,B = gradiente_descendente(W,B,deltas,Y[0:3],alfa,beta)
         W1 = W[0]
         W2 = W[1]
         W3 = W[2]
         b1 = B[0]
         b2 = B[1]
         b3 = B[2]

        #VALIDATION
        for i in range(len(letras_validation)):
         Y = feedforward(letras_validation[i][0],W,B)
         Ye = np.array(letras_validation[i][1])
         err_validation = err_validation + ((Ye-Y[-1])**2)

        err_train_epoc.append(get_mse(np.mean(err_training),len(letras_train)))
        err_valid_epoc.append(get_mse(np.mean(err_validation),len(letras_validation)))


80
10
10
60
20
20
40
30
30
400
50
50
300
100
100
200
150
150
800
100
100
600
200
200
400
300
300


In [44]:
print(len(data_train))
print(len(data_validation))
print(len(data_test))

80
10
10


In [30]:
data = (pd.read_csv(os.path.join(os.path.abspath(''),"data","distorsionadas",cantidad[0],'letras.csv'),sep=';',header=None)).to_numpy()
#Convertimos el dataframe en array numpy
data_train = data[:int(len(data)-int(len(data)*set_validation[percentage])*2)]
data_validation = data[int(len(data)-int(len(data)*set_validation[percentage])*2):int(len(data)-int(len(data)*set_validation[percentage])*2)+cantidad*set_validation[percentage]]
data_test = data[int(len(data)-int(len(data)*set_validation[percentage])*2)+cantidad*set_validation[percentage]:]

TypeError: list indices must be integers or slices, not float

In [ ]:
set =['100','500','1000']
set_validation = [0.1,0.2,0.3]
data=[set,set_validation]
tabla = pd.DataFrame(data,colum=)